In [ ]:
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs


import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-82,central_longitude=20)#260-360)
axex = [165,210,-86,-77]

In [ ]:
def makebackground(ax,axex):
    cmap = plt.get_cmap('ocean')
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(5000,8000),y=slice(7000,10000))
    mask = xr.where(ds.mask==1,2,ds.mask)
    #mask[:] = xr.where(mask==1,2,mask)
    ds = add_lonlat(ds)
    ds.lon[:] = xr.where(ds.lon<0,ds.lon+360,ds.lon)
    ax.set_facecolor(".7")
    
    cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.algae_r')(np.linspace(0,.4,255)))
    cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.2,.6,255)))

    im1 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==0,ds.bed,np.nan),cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
    #im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.surface-ds.thickness,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=0,vmax=2000)
    im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.bed,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

    ax.set_extent(axex,crs=ccrs.PlateCarree())
    #gl = ax.gridlines(linewidth=.2,color='.9')
    #gl.xlocator = mpl.ticker.FixedLocator(np.arange(-180,179,1))
    #gl.ylocator = mpl.ticker.FixedLocator(np.arange(-89,89,.2))

In [ ]:
vmax = 100
vmin = -10
linthresh = .3
linscale = .25

fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
#fracpos = (np.log10(vmax/linthresh)+.5*linscale)/(np.log10(vmax)+np.log10(-(vmin/linthresh))+1*linscale)

#print(fracpos)
nneg = np.int_((1-fracpos)*256)+1

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,1,nneg))
#colors1 = plt.cm.PuBu(np.linspace(.2,.8, nneg))
colors2 = plt.cm.inferno(np.linspace(0, 1, 256-nneg))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)

In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.05,right=.95,top=.95,bottom=.1,wspace=.2,hspace=.1)

ds = xr.open_dataset(f'../../results/Ross_2.0_linear_S134.3_T1-2.3_720.nc')
ds.lon[:] = xr.where(ds.lon<0,ds.lon+360,ds.lon)
fig = plt.figure(figsize=(6,7))
ax = fig.add_subplot(111,projection=proj)
makebackground(ax,axex)
im = ax.pcolormesh(ds.lon,ds.lat,xr.where(ds.tmask==1,ds.melt,np.nan),shading='auto',cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),transform=ccrs.PlateCarree())

#ax.contour(ds.lon,ds.lat,xr.where(ds.tmask==1,ds.melt,np.nan),levels=[0,1e5],colors='w',transform=ccrs.PlateCarree())

ax.gridlines(draw_labels=True)
axins = inset_axes(ax,width="100%",height="5%",loc='lower left',bbox_to_anchor=(0,-.12,1,1),bbox_transform=ax.transAxes,borderpad=0)
ticks = [-3,-1,-.3,-.1,0,.1,.3,1,3,10]
cbar = plt.colorbar(im, cax=axins,extend='both',orientation='horizontal')
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)
cbar.set_label('Freezing / Melt rate [m yr$^{-1}$]')


plt.savefig('../../figures/draftplot_Ross_2.0.png',dpi=450)

In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.05,right=.95,top=.95,bottom=.1,wspace=.2,hspace=.1)

ds = xr.open_dataset(f'../../results/Ross_1.0_linear_S134.3_T1-2.3_050.nc')
ds.lon[:] = xr.where(ds.lon<0,ds.lon+360,ds.lon)
fig = plt.figure(figsize=(6,7))
ax = fig.add_subplot(111,projection=proj)
makebackground(ax,axex)
im = ax.pcolormesh(ds.lon,ds.lat,xr.where(ds.tmask==1,ds.melt,np.nan),shading='auto',cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),transform=ccrs.PlateCarree())

#ax.contour(ds.lon,ds.lat,xr.where(ds.tmask==1,ds.melt,np.nan),levels=[0,1e5],colors='w',transform=ccrs.PlateCarree())

ax.gridlines(draw_labels=True)
axins = inset_axes(ax,width="100%",height="5%",loc='lower left',bbox_to_anchor=(0,-.12,1,1),bbox_transform=ax.transAxes,borderpad=0)
ticks = [-3,-1,-.3,-.1,0,.1,.3,1,3,10]
cbar = plt.colorbar(im, cax=axins,extend='both',orientation='horizontal')
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticks)
cbar.set_label('Freezing / Melt rate [m yr$^{-1}$]')


plt.savefig('../../figures/draftplot_Ross_1.0.png',dpi=450)